In [1]:
import numpy as np 
import pygame 

from formula_functions import *
from animation_functions import *
from intersection_functions import create_spiral, distance
from show_parameters_functions import *
from key_controls_functions import *


from variables import *

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def calc_spiral_coord(t=1 ,v=1, w=1, k=0) -> tuple:
    
    theta_0 = k * np.pi/2

    lin_space = (0, t)
    
    num = 3000

    T = np.linspace(*lin_space, num)
    
    angles = np.array([theta_0 + t * w for t in T])
    radiuses = np.array([v * t for t in T])
    
    x = radiuses * np.cos(angles)
    y = radiuses * np.sin(angles)

    return x, y, T 

In [3]:
# def draw_spiral(spiral_layer, half_screen_width, half_screen_height,length,t=1, v=1,w=1,k=0, t_diagram_mode=False):
    
#     spiral_layer.fill((0, 0, 0, 0))
    

#     if w != 0:
#         x_spiral, y_spiral, T = calc_spiral_coord(t=t ,v=v, w=w, k=k)

#         x_spiral = [x_transform(x, half_screen_width, length) for x in x_spiral]
#         y_spiral = [y_transform(y, half_screen_height, length) for y in y_spiral]

#         for i in range(len(x_spiral) -1):
#             curr_sp_point_x, curr_sp_point_y = x_spiral[i], y_spiral[i]
            
#             if (0 <=abs(curr_sp_point_x)<=half_screen_width * 2) and \
#                (0 <=abs(curr_sp_point_y)<=half_screen_height * 2): 
                
#                 next_sp_point_x, next_sp_point_y = x_spiral[i+1], y_spiral[i+1]
#                 if not t_diagram_mode:
#                     pygame.draw.aalines(spiral_layer, 'red',  False, [(curr_sp_point_x, curr_sp_point_y), \
#                                                                       (next_sp_point_x, next_sp_point_y)])
#                 else:
                 
#                     curr_t_point = T[i]
#                     curr_t_point = x_transform(curr_t_point, half_screen_width, length)
               
#                     next_t_point = T[i+1]
#                     next_t_point= x_transform(next_t_point, half_screen_width, length)
              
#                     backward_curr_sp_x = x_inverse_transform(curr_sp_point_x, half_screen_width, length)
#                     backward_curr_sp_x = y_transform(backward_curr_sp_x, half_screen_height, length)
                    
#                     backward_next_sp_x =  x_inverse_transform(next_sp_point_x, half_screen_width, length)
#                     backward_next_sp_x = y_transform(backward_next_sp_x, half_screen_height, length)
                    
#                     pygame.draw.aalines(spiral_layer, 'red',  False, [(curr_t_point, backward_curr_sp_x), \
#                                                                       (next_t_point, backward_next_sp_x)])
             
#                     pygame.draw.aalines(spiral_layer, 'blue',  False, [(curr_t_point, curr_sp_point_y), \
#                                                                       (next_t_point, next_sp_point_y)])
#         if t_diagram_mode:           
#             start_point = (x_transform(T[0], half_screen_width, length), y_transform(0, half_screen_height, length))
#             end_point = (x_transform(T[-1], half_screen_width, length), y_transform(T[-1] * v, half_screen_height, length))
#             pygame.draw.aalines(spiral_layer, 'green',  False, [start_point, end_point])
                                                                          

In [ ]:
def main_loop(screen_width, screen_height, FPS):

    '''
    Initializations
    '''
    
    pygame.init()

    font_small = pygame.font.Font(None, 26)

    bg_color = (255, 255, 255)
    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer
    background_surface = pygame.Surface((screen_width, screen_height))
   
    ''' Dynamic layers - those that are connected with changeable spiral and line variables '''
    
    line_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    spiral_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    y_axis_intersects_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    line_intersects_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    
    
    ''''Mode layers'''
    # A layer that visualizes the consecutive steps of the algorithm for finding the intersection point between the spiral and the line. 
    algorithm_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    
    '''Data layers'''
    
    # Layer that shows the current status of the spiral and the line parameters 
    params_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA) 
    
    # Shows current status of the switch modes variables
    show_modes_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA) 
    
    show_algorithm_data_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA) 
    
 
    
    t_mth_aproxim_list = [] # caches the already computed approximation values
    
    steps_dict_constants = {'t': t_step, 'v': v_step, 'w': w_step, 'k': k_step, 'x': x_step}
    
    const_params_dict = {'t': t,'v': v, 'w': w, 'k': k, 'x': vert_line_x}
    
    var_params_dict = {'t': t_additional,
                   'v': v_additional, 
                   'w': w_additional, 
                   'k': k_additional, 
                   'x': x_additional}
    
 
    mode_statuses_dict = {'Coordinates':[background_surface, background_mode],
                          'Algorithm mode': [algorithm_layer, algorithm_mode],
                          'Algorithm data mode': [show_algorithm_data_layer, algorithm_data_mode],
                          'T-diagram': [ None, t_diagram_mode],
                          'Line': [line_layer, line_mode],
                          'Spiral': [spiral_layer, spiral_mode],
                          'Y-intersects': [y_axis_intersects_layer, y_axis_intersects],
                          'Line-intersects': [line_intersects_layer, line_intersects],
                          'Parameters': [params_layer, parameters_mode],
                          'Modes layer': [show_modes_layer, True] 
                           }

    algorithm_variables_dict = {'n': 0, 'm': 0, 'total_n': 0}
    
    
    # Create background coordinate system
    create_background(background_surface, screen_width, screen_height, units, length, bg_color, font_small)

    draw_vertical_line(line_layer, half_screen_width, half_screen_height, length, half_units)
    
    draw_spiral(spiral_layer, half_screen_width, half_screen_height,length,t=t, v=v,w=w,k=k,
                t_diagram_mode=mode_statuses_dict['T-diagram'][1])
    
    t_nth_list = calc_y_intersects_t(const_params_dict['t'], const_params_dict['w'], const_params_dict['k'])
      
        
    # Initial drawing of the y-intersection point
    draw_dots(y_axis_intersects_layer, half_screen_width, half_screen_height, length, t_nth_list, v, w, k, 'black')
    
    show_parameters(params_layer, font_small, const_params_dict,
                    var_params_dict)
    
    show_mode_statuses(show_modes_layer, font_small, mode_statuses_dict)
    
    
    # Atach layers to screen
    blit_layers(win, mode_statuses_dict, bg_color)
    
    pygame.display.update()
 
    running = True
    
    while running:
        
        clock.tick(FPS)
        
        update_screen, update_spiral, update_line = False, False, False
      
        # Check and handle for key control events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            else:
                
                
                update_screen, mode_statuses_dict, is_turn_off, is_t_diagram_change  = handle_ctrl_commands(event, update_screen, mode_statuses_dict)
                
                if mode_statuses_dict['Algorithm mode'][1]:
                    
                    algorithm_variables_dict, update_screen = handle_algorithm_mode_controls(event, algorithm_variables_dict, 
                                                                                             t_mth_aproxim_list,update_screen)
                    
                else:
                    update_screen, update_spiral,\
                    const_params_dict, var_params_dict = handle_shift_key_commands(event, update_screen, update_spiral, 
                                                                          const_params_dict, var_params_dict,mode_statuses_dict)
                  
                if is_turn_off is not None:
                    
                    algorithm_layer.fill((0, 0, 0,0))
                    show_algorithm_data_layer.fill((0, 0, 0, 0))
                    t_mth_aproxim_list.clear()
                    algorithm_variables_dict = {'n': 0, 'm': 0, 'total_n': 0}

        # Block the spiral parameters if the program is in algorithm mode.
        if not mode_statuses_dict['Algorithm mode'][1]:
            update_screen, update_spiral, \
            update_line, var_params_dict, \
            const_params_dict = handle_key_commands(const_params_dict, var_params_dict, steps_dict_constants,
                                                    half_screen_width, half_screen_height, length, half_units,
                                                    update_screen, update_spiral, update_line)
                                                    

        if update_screen:

          
            curr_x = const_params_dict['x'] + var_params_dict['x']
            curr_t = const_params_dict['t'] + var_params_dict['t']
            curr_v = const_params_dict['v'] + var_params_dict['v']
            curr_w = const_params_dict['w'] + var_params_dict['w']
            curr_k = const_params_dict['k'] + var_params_dict['k']

            if update_spiral or is_t_diagram_change:
                
                if mode_statuses_dict['T-diagram'][1]:
                    mode_statuses_dict['Line'][1] = False 
                    
                draw_spiral(spiral_layer, half_screen_width, half_screen_height, length,
                                t= curr_t, 
                                v= curr_v,
                                w= curr_w,
                                k= curr_k,
                                t_diagram_mode=mode_statuses_dict['T-diagram'][1])
            if update_line:            
                draw_vertical_line(line_layer, half_screen_width, half_screen_height,
                           length, half_units, x_axis_value=curr_x)

            # A list contains the exact t-parameters for the intersection points of the spiral with y-axis
            t_nth_list = calc_y_intersects_t(curr_t, curr_w, curr_k)
            
            # Draw current y intersection points if y-intersects mode is `on`
            if mode_statuses_dict['Y-intersects'][1]:
                
                draw_dots(y_axis_intersects_layer, half_screen_width, half_screen_height, length, t_nth_list, curr_v, curr_w, curr_k, 'black')

            # A list contains the exact t-parameters for the intersection points of the spiral with the vertical line
            t_mth_list = calc_line_intersections_t(t_nth_list, curr_x,curr_v, curr_w, curr_k, i=400, accuracy=7)
            
            # Draw current line intersection points if line-intersects mode is `on`
            if mode_statuses_dict['Line-intersects'][1]:
                draw_dots(line_intersects_layer, half_screen_width, half_screen_height, length, t_mth_list, curr_v, curr_w, curr_k, 'purple')

            # Show the current statuses of the spiral and line parameters
            show_parameters(params_layer, font_small, const_params_dict, var_params_dict)  
            
            # Show the current statuses of the mode parameters
            show_mode_statuses(show_modes_layer, font_small, mode_statuses_dict)

            
            
            # Show the algorithm steps towards the interesection point
            if mode_statuses_dict['Algorithm mode'][1]:
           
                t_mth_aproxim_list, algorithm_variables_dict = draw_algorithm_steps(algorithm_layer, t_nth_list, 
                                                          curr_v, curr_w, curr_k, curr_x, 
                                                          t_mth_aproxim_list, algorithm_variables_dict,
                                                          half_screen_width, half_screen_height, length,
                                                          accuracy=5)
            
                x, y = 'None', 'None'
                if len(t_mth_aproxim_list) > 1:
         
                    mth_t = t_mth_aproxim_list[algorithm_variables_dict['m']]
                    x, y = calc_single_t_aproxim(curr_v, curr_w, curr_k,  mth_t, half_screen_width, half_screen_height, length, transform=False)
                
                # Show the current statuses of the m_th aproximation point
                show_algorithm_rows_and_cols(show_algorithm_data_layer, x, y, algorithm_variables_dict, font_small)
                
            # Attache the new drawings on the screen    
            blit_layers(win, mode_statuses_dict, bg_color)
            
            pygame.display.update()

        
    # Exit
    pygame.quit()
    
main_loop(screen_width, screen_height, FPS)